> Pandas分组与聚合
> - Written by leiding
>> - athletes.csv： 案例—运动员信息的分组与聚合

# 1 数据分组

In [1]:
import numpy as np
import pandas as pd

## 1.1 通过Series对象进行分组

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': range(5),
                  'data2': range(10, 15)})
df

,key1,key2,data1,data2
0,a,one,0,10
1,a,two,1,11
2,b,one,2,12
3,b,two,3,13
4,a,one,4,14


In [3]:
sr = df['data1']
grouped1 = sr.groupby(df['key1'])
print(type(grouped1))
list(grouped1)

<class 'pandas.core.groupby.generic.SeriesGroupBy'>


[('a',
  0    0
  1    1
  4    4
  Name: data1, dtype: int64),
 ('b',
  2    2
  3    3
  Name: data1, dtype: int64)]

In [4]:
type(grouped1.get_group('a'))

pandas.core.series.Series

In [5]:
grouped1.mean()

key1
a    1.666667
b    2.500000
Name: data1, dtype: float64

In [6]:
grouped2 = df.groupby(df['key1'])
print(type(grouped2))
list(grouped2)

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


[('a',
    key1 key2  data1  data2
  0    a  one      0     10
  1    a  two      1     11
  4    a  one      4     14),
 ('b',
    key1 key2  data1  data2
  2    b  one      2     12
  3    b  two      3     13)]

In [7]:
grouped1x = sr.groupby([df['key1'], df['key2']])  # 多个分组键
list(grouped1x)

[(('a', 'one'),
  0    0
  4    4
  Name: data1, dtype: int64),
 (('a', 'two'),
  1    1
  Name: data1, dtype: int64),
 (('b', 'one'),
  2    2
  Name: data1, dtype: int64),
 (('b', 'two'),
  3    3
  Name: data1, dtype: int64)]

In [8]:
grouped1x.get_group(('a', 'one'))

0    0
4    4
Name: data1, dtype: int64

In [9]:
grouped1x.mean()

key1  key2
a     one     2.0
      two     1.0
b     one     2.0
      two     3.0
Name: data1, dtype: float64

In [10]:
# 使用Series进行分组
df.groupby(pd.Series(['a', 'a', 'b', 'b', 'a'])).mean()

,data1,data2
a,1.666667,11.666667
b,2.500000,12.500000


In [11]:
list(df.groupby(pd.Series(['one', 'one', 'two'])))  # 任意Series

[('one',
    key1 key2  data1  data2
  0    a  one      0     10
  1    a  two      1     11),
 ('two',
    key1 key2  data1  data2
  2    b  one      2     12)]

In [12]:
list(df.groupby([pd.Series(['A', 'A', 'B', 'B']), pd.Series([1, 2, 1, 2])]))

[(('A', 1.0),
    key1 key2  data1  data2
  0    a  one      0     10),
 (('A', 2.0),
    key1 key2  data1  data2
  1    a  two      1     11),
 (('B', 1.0),
    key1 key2  data1  data2
  2    b  one      2     12),
 (('B', 2.0),
    key1 key2  data1  data2
  3    b  two      3     13)]

## 1.2 通过列名进行分组

In [13]:
list(df.groupby('key1'))

[('a',
    key1 key2  data1  data2
  0    a  one      0     10
  1    a  two      1     11
  4    a  one      4     14),
 ('b',
    key1 key2  data1  data2
  2    b  one      2     12
  3    b  two      3     13)]

In [14]:
list(df.groupby(['key1', 'key2']))

[(('a', 'one'),
    key1 key2  data1  data2
  0    a  one      0     10
  4    a  one      4     14),
 (('a', 'two'),
    key1 key2  data1  data2
  1    a  two      1     11),
 (('b', 'one'),
    key1 key2  data1  data2
  2    b  one      2     12),
 (('b', 'two'),
    key1 key2  data1  data2
  3    b  two      3     13)]

In [15]:
print(df.groupby(['key1', 'key2']).size())  # 每个组的大小
df.groupby(['key1', 'key2']).count()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64


data1  data2
key1 key2              
a    one       2      2
     two       1      1
b    one       1      1
     two       1      1

## 1.3 分组对象

In [16]:
# 遍历GroupBy对象
for name, group in df.groupby('key1'):
    print(name)
    print(group, '\n')

a
  key1 key2  data1  data2
0    a  one      0     10
1    a  two      1     11
4    a  one      4     14 

b
  key1 key2  data1  data2
2    b  one      2     12
3    b  two      3     13 



In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group, '\n')

a one
  key1 key2  data1  data2
0    a  one      0     10
4    a  one      4     14 

a two
  key1 key2  data1  data2
1    a  two      1     11 

b one
  key1 key2  data1  data2
2    b  one      2     12 

b two
  key1 key2  data1  data2
3    b  two      3     13 



In [18]:
# 仅聚合一列
print(df.groupby('key1')['data2'].mean())
df['data2'].groupby(df['key1']).mean()

key1
a    11.666667
b    12.500000
Name: data2, dtype: float64


key1
a    11.666667
b    12.500000
Name: data2, dtype: float64

In [19]:
print(df[['data2']].groupby([df['key1'], df['key2']]).mean())
df.groupby(['key1', 'key2'])[['data2']].mean()

           data2
key1 key2       
a    one    12.0
     two    11.0
b    one    12.0
     two    13.0


data2
key1 key2       
a    one    12.0
     two    11.0
b    one    12.0
     two    13.0

## 1.4 其他分组方式

In [20]:
# 对列进行分组
for name, group in df.groupby(['first', 'first', 'second', 'second'], axis=1):
    print(name)
    print(group, '\n')

first
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one 

second
   data1  data2
0      0     10
1      1     11
2      2     12
3      3     13
4      4     14 



In [21]:
# 通过字典进行分组
num_df = pd.DataFrame({'a': [1, 2, 3, 4, 5], 
                      'b': [6, 7, 8, 9, 10],
                      'c': [11, 12, 13, 14, 15],
                      'd': [5, 4, 3, 2, 1],
                      'e': [10, 9, 8, 7, 6]})
print(num_df, '\n')
for i in num_df.groupby({'a': 'first', 'b': 'first', 'c': 'second',
                     'd': 'second', 'e': 'second'}, axis=1):
    print(i, '\n')

   a   b   c  d   e
0  1   6  11  5  10
1  2   7  12  4   9
2  3   8  13  3   8
3  4   9  14  2   7
4  5  10  15  1   6 

('first',    a   b
0  1   6
1  2   7
2  3   8
3  4   9
4  5  10) 

('second',     c  d   e
0  11  5  10
1  12  4   9
2  13  3   8
3  14  2   7
4  15  1   6) 



In [22]:
# 通过函数进行分组
df = pd.DataFrame({'a': [1, 2, 3, 4, 5],
                   'b': [6, 7, 8, 9, 10],
                   'c': [5, 4, 3, 2, 1]},
                  index=['Sun', 'Jack', 'Alice', 'Helen', 'Job'])
print(df, '\n')
for group in df.groupby(len):
    print(group, '\n')

       a   b  c
Sun    1   6  5
Jack   2   7  4
Alice  3   8  3
Helen  4   9  2
Job    5  10  1 

(3,      a   b  c
Sun  1   6  5
Job  5  10  1) 

(4,       a  b  c
Jack  2  7  4) 

(5,        a  b  c
Alice  3  8  3
Helen  4  9  2) 



# 2 数据聚合

## 2.1 使用内置统计方法聚合

In [23]:
df = pd.DataFrame({'key1': ['A', 'A', 'B', 'B', 'A'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   "data1": [2, 3, 4, 6, 8],
                   "data2": [3, 5, np.nan, 3,7]})
df

,key1,key2,data1,data2
0,A,one,2,3.0
1,A,two,3,5.0
2,B,one,4,NaN
3,B,two,6,3.0
4,A,one,8,7.0


In [24]:
df.groupby('key1').mean()

,data1,data2
key1,,
A,4.333333,5.0
B,5.000000,3.0


In [25]:
df.groupby('key1').describe()

data1                                              data2                 \
     count      mean       std  min  25%  50%  75%  max count mean  std  min   
key1                                                                           
A      3.0  4.333333  3.214550  2.0  2.5  3.0  5.5  8.0   3.0  5.0  2.0  3.0   
B      2.0  5.000000  1.414214  4.0  4.5  5.0  5.5  6.0   1.0  3.0  NaN  3.0   

                          
      25%  50%  75%  max  
key1                      
A     4.0  5.0  6.0  7.0  
B     3.0  3.0  3.0  3.0

## 2.2 面向列的聚合

In [26]:
data_frame = pd.DataFrame(np.arange(36).reshape((6, 6)),
                       columns=list('ABCDEF'))
data_frame['key'] = pd.Series(list('aaabbb'), name='key')
data_frame

,A,B,C,D,E,F,key
0,0,1,2,3,4,5,a
1,6,7,8,9,10,11,a
2,12,13,14,15,16,17,a
3,18,19,20,21,22,23,b
4,24,25,26,27,28,29,b
5,30,31,32,33,34,35,b


In [27]:
data_group = data_frame.groupby('key')
data_group.get_group('a')

,A,B,C,D,E,F,key
0,0,1,2,3,4,5,a
1,6,7,8,9,10,11,a
2,12,13,14,15,16,17,a


In [28]:
dict([x for x in data_group])['b']

,A,B,C,D,E,F,key
3,18,19,20,21,22,23,b
4,24,25,26,27,28,29,b
5,30,31,32,33,34,35,b


In [29]:
data_group.agg(sum)

,A,B,C,D,E,F
key,,,,,,
a,18,21,24,27,30,33
b,72,75,78,81,84,87


In [30]:
def range_data_group(arr):
    return arr.max()-arr.min()
data_group.agg(range_data_group) 

,A,B,C,D,E,F
key,,,,,,
a,12,12,12,12,12,12
b,12,12,12,12,12,12


## 2.3 使用多函数聚合

In [31]:
# data_group.agg([range_data_group, sum])
data_group.agg([range_data_group, 'sum'])

A                    B                    C      \
    range_data_group sum range_data_group sum range_data_group sum   
key                                                                  
a                 12  18               12  21               12  24   
b                 12  72               12  75               12  78   

                   D                    E                    F      
    range_data_group sum range_data_group sum range_data_group sum  
key                                                                 
a                 12  27               12  30               12  33  
b                 12  81               12  84               12  87

In [32]:
data_group.agg([("极差", range_data_group), ("和", 'sum')])

A       B       C       D       E       F    
     极差   和  极差   和  极差   和  极差   和  极差   和  极差   和
key                                                
a    12  18  12  21  12  24  12  27  12  30  12  33
b    12  72  12  75  12  78  12  81  12  84  12  87

In [33]:
data_group.agg({'A': 'sum', 'B': 'mean', 'C': range_data_group})

,A,B,C
key,,,
a,18,7.0,12
b,72,25.0,12


# 3 其他分组级运算

## 3.1 数据转换

In [34]:
df = pd.DataFrame({'a': [0, 1, 6, 10, 3],
                   'b': [1, 2, 7, 11, 4],
                   'c': [2, 3, 8, 12, 4],
                   'd': [3, 4, 9, 13, 5],
                   'e': [4, 5, 10, 14, 3],
                   'key': ['A', 'A', 'B', 'B', 'B']})
df

,a,b,c,d,e,key
0,0,1,2,3,4,A
1,1,2,3,4,5,A
2,6,7,8,9,10,B
3,10,11,12,13,14,B
4,3,4,4,5,3,B


In [35]:
print(df.groupby('key').mean())
df.groupby('key').transform('mean')

            a         b    c    d    e
key                                   
A    0.500000  1.500000  2.5  3.5  4.5
B    6.333333  7.333333  8.0  9.0  9.0


,a,b,c,d,e
0,0.500000,1.500000,2.5,3.5,4.5
1,0.500000,1.500000,2.5,3.5,4.5
2,6.333333,7.333333,8.0,9.0,9.0
3,6.333333,7.333333,8.0,9.0,9.0
4,6.333333,7.333333,8.0,9.0,9.0


In [36]:
df = pd.DataFrame({'A': [2, 3, 3, 4, 2],
                   'B': [4, 2, 3, 6, 6],
                   'C': [9, 7, 0, 7, 8],
                   'D': [3, 4, 8, 6, 10]})
df

,A,B,C,D
0,2,4,9,3
1,3,2,7,4
2,3,3,0,8
3,4,6,7,6
4,2,6,8,10


In [37]:
key = ['one','one','two',' two',' two']
df.groupby(key).transform('mean')

,A,B,C,D
0,2.5,3.0,8.0,3.5
1,2.5,3.0,8.0,3.5
2,3.0,3.0,0.0,8.0
3,3.0,6.0,7.5,8.0
4,3.0,6.0,7.5,8.0


In [38]:
df = pd.DataFrame(np.random.randn(5, 4))
df.groupby(key).transform(lambda x: x - x.mean())

,0,1,2,3
0,0.246218,-0.797014,-0.964961,-0.376341
1,-0.246218,0.797014,0.964961,0.376341
2,0.000000,0.000000,0.000000,0.000000
3,-0.468330,0.219168,0.590368,-1.414103
4,0.468330,-0.219168,-0.590368,1.414103


## 3.2 数据应用

In [39]:
data_frame = pd.DataFrame({'data1': [80,23,25,63,94,92,99,92,82,99],
                           'data2': [41,87,58,68,72,89,60,42,53,65],
                           'data3': [30,78,23,66,16,59,20,23,24,40],
                           'key': list('baabbabaaa')})
data_frame

,data1,data2,data3,key
0,80,41,30,b
1,23,87,78,a
2,25,58,23,a
3,63,68,66,b
4,94,72,16,b
5,92,89,59,a
6,99,60,20,b
7,92,42,23,a
8,82,53,24,a
9,99,65,40,a


In [40]:
data_by_group = data_frame.groupby('key')
dict([x for x in data_by_group])['a']

,data1,data2,data3,key
1,23,87,78,a
2,25,58,23,a
5,92,89,59,a
7,92,42,23,a
8,82,53,24,a
9,99,65,40,a


In [41]:
dict([x for x in data_by_group])['b']

,data1,data2,data3,key
0,80,41,30,b
3,63,68,66,b
4,94,72,16,b
6,99,60,20,b


In [42]:
data_by_group.apply(max)  # 替代聚合

,data1,data2,data3,key
key,,,,
a,99,89,78,a
b,99,72,66,b


# 4 拆分 - 应用 - 合并的示例

## 4.1 分组加权平均

In [43]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                               'b', 'b', 'b', 'b'],
                  'data': np.random.randn(8),
                  'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.197283,0.128881
1,a,-0.154572,0.716419
2,a,0.081539,0.713065
3,a,1.480634,0.765769
4,b,-0.780914,0.346373
5,b,-0.992327,0.102204
6,b,-0.926082,0.047732
7,b,-0.642090,0.209466


In [44]:
df.groupby('category').apply(lambda g: np.average(g['data'], weights=g['weights']))

category
a    0.476158
b   -0.780146
dtype: float64

## 4.2 分组填充缺失值

In [45]:
data = pd.Series(np.random.randn(8),
                index=['Beijing', 'Tianjin', 'Jinan', 'Dalian',
                      'Shanghai', 'Guangzhou', 'Shenzhen', 'Wuhan'])
data[['Jinan', 'Guangzhou', 'Wuhan']] = np.nan
data

Beijing     -0.310269
Tianjin     -0.692978
Jinan             NaN
Dalian       0.077597
Shanghai     1.105618
Guangzhou         NaN
Shenzhen    -0.021803
Wuhan             NaN
dtype: float64

In [46]:
# 使用整体平均值填充
data.fillna(data.mean())

Beijing     -0.310269
Tianjin     -0.692978
Jinan        0.031633
Dalian       0.077597
Shanghai     1.105618
Guangzhou    0.031633
Shenzhen    -0.021803
Wuhan        0.031633
dtype: float64

In [47]:
group_key = ['North'] * 4 + ['South'] * 4
data.groupby(group_key).mean()

North   -0.308550
South    0.541907
dtype: float64

In [48]:
# 使用分组的平均值填充
data.groupby(group_key).apply(lambda g: g.fillna(g.mean()))

Beijing     -0.310269
Tianjin     -0.692978
Jinan       -0.308550
Dalian       0.077597
Shanghai     1.105618
Guangzhou    0.541907
Shenzhen    -0.021803
Wuhan        0.541907
dtype: float64

In [49]:
# 分组内使用指定的值填充
fill_values = {'North': 0.5, 'South': -1}
data.groupby(group_key).apply(lambda g: g.fillna(fill_values[g.name]))

Beijing     -0.310269
Tianjin     -0.692978
Jinan        0.500000
Dalian       0.077597
Shanghai     1.105618
Guangzhou   -1.000000
Shenzhen    -0.021803
Wuhan       -1.000000
dtype: float64

## 4.3 分组随机抽样

In [50]:
suits = ['Heart', 'Spade', 'Club', 'Diamond']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'Q', 'K']
cards = []
for suit in suits:
    cards.extend(suit + ' ' + str(num) for num in base_names)
deck = pd.Series(card_val, index=cards)
deck

Heart A        1
Heart 2        2
Heart 3        3
Heart 4        4
Heart 5        5
Heart 6        6
Heart 7        7
Heart 8        8
Heart 9        9
Heart 10      10
Heart J       10
Heart Q       10
Heart K       10
Spade A        1
Spade 2        2
Spade 3        3
Spade 4        4
Spade 5        5
Spade 6        6
Spade 7        7
Spade 8        8
Spade 9        9
Spade 10      10
Spade J       10
Spade Q       10
Spade K       10
Club A         1
Club 2         2
Club 3         3
Club 4         4
Club 5         5
Club 6         6
Club 7         7
Club 8         8
Club 9         9
Club 10       10
Club J        10
Club Q        10
Club K        10
Diamond A      1
Diamond 2      2
Diamond 3      3
Diamond 4      4
Diamond 5      5
Diamond 6      6
Diamond 7      7
Diamond 8      8
Diamond 9      9
Diamond 10    10
Diamond J     10
Diamond Q     10
Diamond K     10
dtype: int64

In [51]:
# 随机抽取n张牌
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

Heart 2       2
Diamond 9     9
Heart J      10
Diamond 6     6
Club 3        3
dtype: int64

In [52]:
# 每种花色随机抽取n张牌
deck.groupby(lambda card: card.split(' ')[0], group_keys=False).apply(draw, n=2)

Club J       10
Club 5        5
Diamond J    10
Diamond Q    10
Heart J      10
Heart A       1
Spade 2       2
Spade J      10
dtype: int64

# 5	案例—运动员信息的分组与聚合

In [53]:
df = pd.read_csv('athletes.csv')
df

,姓名,性别,出生年份（年）,年龄（岁）,身高(cm),体重(kg),项目,省份
0,陈楠,女,1983年,35,197,90,篮球,山东省
1,白发全,男,1986年,32,175,64,铁人三项,云南省
2,陈晓佳,女,1988年,30,180,70,篮球,江苏省
3,陈倩,女,1987年,31,163,54,女子现代五项,江苏省
4,曹忠荣,男,1981年,37,180,73,男子现代五项,上海市
...,...,...,...,...,...,...,...,...
174,赵芸蕾,女,1986年,32,173,62,羽毛球,湖北省
175,周琦,男,1996年,22,217,95,篮球,河南省
176,翟晓川,男,1993年,25,204,100,篮球,河北省
177,赵继伟,男,1995年,23,185,77,篮球,辽宁省


In [54]:
# 按项目分组
df_basketball = dict([x for x in df.groupby('项目')])['篮球']
df_basketball

,姓名,性别,出生年份（年）,年龄（岁）,身高(cm),体重(kg),项目,省份
0,陈楠,女,1983年,35,197,90,篮球,山东省
2,陈晓佳,女,1988年,30,180,70,篮球,江苏省
16,丁彦雨航,男,1993年,25,200,91,篮球,新疆维吾尔自治区
23,高颂,女,1992年,26,191,85,篮球,黑龙江省
28,郭艾伦,男,1993年,25,192,85,篮球,辽宁省
35,黄红枇,女,1989年,29,195,80,篮球,广西壮族自治区
42,黄思静,女,1996年,22,192,80,篮球,广东省
48,李慕豪,男,1992年,26,225,111,篮球,贵州贵阳
54,李珊珊,女,1987年,31,177,70,篮球,江苏省
73,露雯,女,1990年,28,191,78,篮球,内蒙古自治区


In [55]:
# 按性别进行分组
groupby_sex = df_basketball.groupby('性别')
groupby_sex.mean()

,年龄（岁）,身高(cm),体重(kg)
性别,,,
女,28.000000,189.600000,77.900000
男,25.272727,205.090909,97.727273


In [56]:
# 使用transform方法广播
groupby_sex[['年龄（岁）', '身高(cm)', '体重(kg)']].transform('mean')

,年龄（岁）,身高(cm),体重(kg)
0,28.000000,189.600000,77.900000
2,28.000000,189.600000,77.900000
16,25.272727,205.090909,97.727273
23,28.000000,189.600000,77.900000
28,25.272727,205.090909,97.727273
35,28.000000,189.600000,77.900000
42,28.000000,189.600000,77.900000
48,25.272727,205.090909,97.727273
54,28.000000,189.600000,77.900000
73,28.000000,189.600000,77.900000


In [57]:
# 查看男篮运动员的分组
baseketball_male = dict([x for x in groupby_sex])['男']
baseketball_male

,姓名,性别,出生年份（年）,年龄（岁）,身高(cm),体重(kg),项目,省份
16,丁彦雨航,男,1993年,25,200,91,篮球,新疆维吾尔自治区
28,郭艾伦,男,1993年,25,192,85,篮球,辽宁省
48,李慕豪,男,1992年,26,225,111,篮球,贵州贵阳
106,睢冉,男,1992年,26,192,95,篮球,山西省
124,王哲林,男,1994年,24,214,110,篮球,福建省
155,易建联,男,1987年,31,213,113,篮球,广东省
161,周鹏,男,1989年,29,206,90,篮球,辽宁省
175,周琦,男,1996年,22,217,95,篮球,河南省
176,翟晓川,男,1993年,25,204,100,篮球,河北省
177,赵继伟,男,1995年,23,185,77,篮球,辽宁省


In [58]:
# 求年龄、身高、体重这三列数据的极差值
def range_data_group(arr):
    return arr.max()-arr.min()
baseketball_male.agg({'年龄（岁）':range_data_group,
                      '身高(cm)':range_data_group,
                      '体重(kg)':range_data_group})

年龄（岁）      9
身高(cm)    40
体重(kg)    36
dtype: int64

In [59]:
# 添加“体质指数”列
df_basketball['体质指数'] = 0
df_basketball

,姓名,性别,出生年份（年）,年龄（岁）,身高(cm),体重(kg),项目,省份,体质指数
0,陈楠,女,1983年,35,197,90,篮球,山东省,0
2,陈晓佳,女,1988年,30,180,70,篮球,江苏省,0
16,丁彦雨航,男,1993年,25,200,91,篮球,新疆维吾尔自治区,0
23,高颂,女,1992年,26,191,85,篮球,黑龙江省,0
28,郭艾伦,男,1993年,25,192,85,篮球,辽宁省,0
35,黄红枇,女,1989年,29,195,80,篮球,广西壮族自治区,0
42,黄思静,女,1996年,22,192,80,篮球,广东省,0
48,李慕豪,男,1992年,26,225,111,篮球,贵州贵阳,0
54,李珊珊,女,1987年,31,177,70,篮球,江苏省,0
73,露雯,女,1990年,28,191,78,篮球,内蒙古自治区,0


In [60]:
# 计算BMI值
def outer(num):
    def ath_bmi(sum_bmi):
        weight = df_basketball['体重(kg)']
        height = df_basketball['身高(cm)'] 
        sum_bmi =  weight / (height/100)**2
        return num + sum_bmi
    return ath_bmi
all_bmi = df_basketball['体质指数']
df_basketball['体质指数'] = df_basketball[['体质指数']].apply(outer(all_bmi))
df_basketball

,姓名,性别,出生年份（年）,年龄（岁）,身高(cm),体重(kg),项目,省份,体质指数
0,陈楠,女,1983年,35,197,90,篮球,山东省,23.190497
2,陈晓佳,女,1988年,30,180,70,篮球,江苏省,21.604938
16,丁彦雨航,男,1993年,25,200,91,篮球,新疆维吾尔自治区,22.750000
23,高颂,女,1992年,26,191,85,篮球,黑龙江省,23.299800
28,郭艾伦,男,1993年,25,192,85,篮球,辽宁省,23.057726
35,黄红枇,女,1989年,29,195,80,篮球,广西壮族自治区,21.038790
42,黄思静,女,1996年,22,192,80,篮球,广东省,21.701389
48,李慕豪,男,1992年,26,225,111,篮球,贵州贵阳,21.925926
54,李珊珊,女,1987年,31,177,70,篮球,江苏省,22.343516
73,露雯,女,1990年,28,191,78,篮球,内蒙古自治区,21.380993
